In [ ]:
import os 
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
print('asdas')

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from keras.layers import Reshape

In [4]:
# Define paths and parameters
data_dir = './CelebA'
image_dir = os.path.join(data_dir, 'img_align_celeba')
anno_path = os.path.join(data_dir, 'list_attr_celeba.txt')
image_size = 64

In [5]:
# Load annotations
df = pd.read_csv(anno_path, sep=' ', skipinitialspace=True)
df = df.replace(to_replace=-1, value=0)  # replace -1 with 0
df = df.sample(frac=1).reset_index(drop=True)  # shuffle

In [6]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
# Define helper functions
def load_image(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    img = img.resize((image_size, image_size))
    return img


def preprocess_data(df):
    X = []
    y = []
    for i, row in df.iterrows():
        image_path = os.path.join(image_dir, f"{row.name}.jpg")
        try:
            img = load_image(image_path)
        except: 
            continue
        X.append(np.array(img))
        y.append(row[1:])
    X = np.array(X) / 255.0  # normalize pixel values
    y = np.array(y)
    return X, y

# Preprocess data
X_train, y_train = preprocess_data(train_df)
X_test, y_test = preprocess_data(test_df)

num_classes = y_train.shape[1]
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [28]:
# Construct the model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    Flatten(),
])
model.add(Dense(256, activation='relu'))
model.add(Dense(1521, activation='sigmoid'))
model.add(Reshape((39, 39)))

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [29]:
print(model.summary())
print(y_test.shape)
print(y_train.shape)
print(X_test.shape)
print(X_train.shape)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 12, 12, 64)        36928     
                                                                 
 flatten_6 (Flatten)         (None, 9216)             

In [30]:
# Train the model
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=10,
                    validation_data=(X_test, y_test),
                    callbacks=[csv_logger])

Epoch 1/10
1282/1282 [==============================] - 47s 35ms/step - loss: 0.4835 - accuracy: 0.7958 - val_loss: 0.4316 - val_accuracy: 0.8020
Epoch 2/10
1282/1282 [==============================] - 37s 29ms/step - loss: 0.4294 - accuracy: 0.8033 - val_loss: 0.4309 - val_accuracy: 0.8020
Epoch 3/10
1282/1282 [==============================] - 37s 29ms/step - loss: 0.4292 - accuracy: 0.8033 - val_loss: 0.4308 - val_accuracy: 0.8020
Epoch 4/10
1282/1282 [==============================] - 37s 29ms/step - loss: 0.4291 - accuracy: 0.8033 - val_loss: 0.4306 - val_accuracy: 0.8020
Epoch 5/10
1282/1282 [==============================] - 37s 29ms/step - loss: 0.4291 - accuracy: 0.8033 - val_loss: 0.4307 - val_accuracy: 0.8020
Epoch 6/10
1282/1282 [==============================] - 36s 28ms/step - loss: 0.4290 - accuracy: 0.8033 - val_loss: 0.4306 - val_accuracy: 0.8020
Epoch 7/10
1282/1282 [==============================] - 36s 28ms/step - loss: 0.4290 - accuracy: 0.8033 - val_loss: 0.4307 -

In [35]:
# Evaluate the model
from sklearn.metrics import recall_score, f1_score
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('recall', recall_score(y_train,y_test))
print('f1_score', f1)

643/643 [==============================] - 6s 10ms/step - loss: 0.4306 - accuracy: 0.8020
Test accuracy: 0.8019599914550781


ValueError: Found input variables with inconsistent numbers of samples: [82023, 20576]